## 2.2 Random Forest
In this task you are suppose to implement 2 types of decision trees: 1. Using only Python. 2. Using random forest with a library such as sklearn.
The classification should be to predict recurrent cancer.
* Download the Breast Cancer dataset https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.data
* Predict the two classes: Not-recurrent and recurrent
* Implement and test a decision tree from scratch using Python and standard libraries.
* Implement and test random forest with a library such as sklearn.
* Choose the network architecture with care.
* Train and validate all algorithms.
* Make the necessary assumptions.
* You can be in groups of up to 3.
* Handin: One page report to be delivered at the end of the semester.


## Get and prepare data
* Number of Instances: 286
* Number of Attributes: 9 + the class attribute
* Attribute Information:<br>
   1\. Class: no-recurrence-events, recurrence-events<br>
   2\. age: 10-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80-89, 90-99.<br>
   3\. menopause: lt40, ge40, premeno.<br>
   4\. tumor-size: 0-4, 5-9, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40-44, 45-49, 50-54, 55-59.<br>
   5\. inv-nodes: 0-2, 3-5, 6-8, 9-11, 12-14, 15-17, 18-20, 21-23, 24-26, 27-29, 30-32, 33-35, 36-39.<br>
   6\. node-caps: yes, no.<br>
   7\. deg-malig: 1, 2, 3.<br>
   8\. breast: left, right.<br>
   9\. breast-quad: left-up, left-low, right-up, right-low, central.<br>
  10\. irradiat: yes, no.<br>
* Missing Attribute Values: (denoted by "?")
   Attribute #:  Number of instances with missing values:<br>
   6\.             8<br>
   9\.             1<br>
* Class Distribution:<br>
    1\. no-recurrence-events: 201 instances<br>
    2\. recurrence-events: 85 instances<br>

In [1]:
import requests
import random
from pprint import pprint

# Get data
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer/breast-cancer.data"
r = requests.get(data_url)

# Split response into lists of lists and remove incomplete rows
dataset = [line.split(',') for line in r.text.splitlines() if '?' not in line]

# Convert 'no-recurrence-events' to 1 and 'recurrence-events' to 0
dataset = [[1 if row[0]=='no-recurrence-events' else 0] + row[1:] for row in dataset]

# Shuffle data
random.seed(2411)
random.shuffle(dataset)

print("Complete dataset contains {:n} rows.".format(len(dataset)))

training_dataset = dataset[:int(len(dataset)*(0.8))]
print("Training dataset (80%) contains {:n} rows.".format(len(training_dataset)))

verification_dataset = dataset[int(len(dataset)*(0.8)):]
print("Verification dataset (20%) contains {:n} rows.".format(len(verification_dataset)))

print("Sample data:")
pprint(dataset[:10], width=120,compact=True)

Complete dataset contains 277 rows.
Training dataset (80%) contains 221 rows.
Verification dataset (20%) contains 56 rows.
Sample data:
[[1, '50-59', 'ge40', '20-24', '0-2', 'no', '1', 'right', 'left_low', 'no'],
 [0, '30-39', 'premeno', '0-4', '0-2', 'no', '2', 'right', 'central', 'no'],
 [1, '60-69', 'ge40', '15-19', '0-2', 'no', '2', 'left', 'left_low', 'no'],
 [1, '30-39', 'premeno', '30-34', '0-2', 'no', '3', 'left', 'left_low', 'no'],
 [1, '60-69', 'ge40', '45-49', '6-8', 'yes', '3', 'left', 'central', 'no'],
 [1, '50-59', 'lt40', '30-34', '0-2', 'no', '3', 'right', 'left_up', 'no'],
 [0, '30-39', 'premeno', '25-29', '6-8', 'yes', '3', 'left', 'right_low', 'yes'],
 [1, '50-59', 'premeno', '15-19', '0-2', 'no', '2', 'right', 'right_low', 'no'],
 [1, '60-69', 'ge40', '10-14', '0-2', 'no', '2', 'right', 'left_up', 'yes'],
 [0, '40-49', 'premeno', '30-34', '12-14', 'yes', '3', 'left', 'left_up', 'yes']]


In [4]:
import math

def split(data,attribute,remove=False):
    retvals = {}
    allattributes = set([i[attribute] for i in data])
    for d in data:
        c = d[attribute]
        aList = retvals.get(c,[])
        if(remove):
            d.pop(attribute)
        aList.append(d)
        retvals[c] = aList
    return retvals

def entropy(oneclass):
    pos = len([i for i in oneclass if i[0]==1])
    neg = len([i for i in oneclass if i[0]==0])
    total = pos+neg
    if(min((pos,neg))==0):
        return 0
    entropy = - (pos/total)*math.log(pos/total,2) - (neg/total)*math.log(neg/total,2)
    return entropy

def gain(oneclass,attribute):
    d = [(entropy(i),len(i)) for i in split(oneclass,attribute,False).values()]
    nAll = sum([i[1] for i in d])
    gain = sum([(i[0]*i[1])/nAll for i in d])
    return gain

def getHighestGain(oneclass):
    before = entropy(oneclass)
    classes = [i for i in range(1,len(oneclass[0]))]
    entropies = [gain(oneclass,c) for c in classes]
    if (all(v == 0 for v in entropies)):
        return 0
    return entropies.index(min(entropies))+1

def isPure(oneclass):
    classes = [i for i in range(1,len(oneclass[0]))]
    
    for c in classes:
        if(len(set([i[c] for i in oneclass]))>1):
            return False
    return True

def isEmpty(oneclass):
    return len(oneclass[0])<=1

def mostCommon(oneclass):
    lst = [i[0] for i in oneclass]
    return max(set(lst), key=lst.count)

def confidence(oneclass):
    mostcommon = mostCommon(oneclass)
    return len([i[0] for i in oneclass if i[0]==mostcommon])/len(oneclass)

actualClassifier = "def classify(data):"

def buildTree(oneclass,spaces="  "):
    global actualClassifier
    highest = getHighestGain(oneclass)
    if(isEmpty(oneclass) or isPure(oneclass) or highest == 0):
        actualClassifier += "\n"+spaces+"return ("+ str(mostCommon(oneclass)) +")"
        return 
    d = split(oneclass,highest)
    for key,value in d.items():
        actualClassifier += "\n"+spaces+"if(data["+str(highest)+"]==\""+str(key)+"\"):"
        buildTree(value,spaces+"  ")
        

## Training

In [6]:
buildTree(training_dataset)
print(actualClassifier)
exec(actualClassifier)

def classify(data):
  if(data[4]=="0-2"):
    if(data[3]=="20-24"):
      if(data[9]=="no"):
        if(data[8]=="left_low"):
          if(data[6]=="1"):
            return (1)
          if(data[6]=="3"):
            if(data[1]=="40-49"):
              return (1)
            if(data[1]=="60-69"):
              return (0)
          if(data[6]=="2"):
            return (0)
        if(data[8]=="left_up"):
          if(data[2]=="premeno"):
            return (1)
          if(data[2]=="ge40"):
            if(data[1]=="50-59"):
              if(data[6]=="3"):
                return (1)
              if(data[6]=="2"):
                if(data[7]=="left"):
                  return (0)
                if(data[7]=="right"):
                  return (1)
            if(data[1]=="60-69"):
              return (1)
            if(data[1]=="40-49"):
              return (0)
        if(data[8]=="right_low"):
          return (1)
        if(data[8]=="central"):
          if(data[1]=="50-59"):
           

## Accuracy

In [7]:
correct,wrong = 0,0
for data in verification_dataset:
    if(data[0]==classify(data)):
        correct+=1
    else:
        wrong+=1
print ("Correct classifications",correct)
print ("Wrong classifications",wrong)
print ("Accuracy",correct/(correct+wrong))


Correct classifications 33
Wrong classifications 23
Accuracy 0.5892857142857143
